# Report on weather data

In [ ]:
import sys
import os
import pandas as pd

In [13]:
# git directories

# Josh
gitdir = "/Users/joshd/Documents/GitHub/GA2024_1/"

#Magda
#gitdir =

# Add the directory
module_path = os.path.abspath(gitdir)
sys.path.append(module_path)

from home_messages_db import HomeMessagesDB

db = HomeMessagesDB('sqlite:///myhome.db')

df_st = db.get_smartthings()
df_p1e = db.get_p1e()
df_p1g = db.get_p1g()
df_weather = db.get_weather_data()




In [9]:
# get overview

# gas data
print(df_p1g.shape)

# energy usage (smartthings)
print(df_st.shape)

# weather
print(df_weather.shape)

(80716, 3)
(1182210, 9)
(20424, 9)


In [32]:
df_ins_temp = df_st[df_st["capability"] == "temperatureMeasurement"]

In [33]:
# data types
df_ins_temp['value'] = df_ins_temp['value'].astype(float)

# rename temperature
df_ins_temp.rename(columns={'value': 'temperature'}, inplace=True)

/var/folders/z1/762t94_j4tx1bhtk_g1t9qh80000gn/T/ipykernel_21265/2229040446.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ins_temp['value'] = df_ins_temp['value'].astype(float)
/var/folders/z1/762t94_j4tx1bhtk_g1t9qh80000gn/T/ipykernel_21265/2229040446.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ins_temp.rename(columns={'value': 'temperature'}, inplace=True)


## Get daily inside temperature average for smartthings

In [34]:
# Convert epoch_seconds to datetime
df_ins_temp['datetime'] = pd.to_datetime(df_ins_temp['epoch'], unit='s')

# Select only the target column (temperature) and the epoch time
df_target = df_ins_temp[['datetime', 'temperature']]

# Set datetime as index
df_target.set_index('datetime', inplace=True)

# Resample by day and calculate the average temperature
df_daily_inside = df_target.resample('D').mean()

# Reset index to make 'datetime' a column again
df_daily_inside.reset_index(inplace=True)

/var/folders/z1/762t94_j4tx1bhtk_g1t9qh80000gn/T/ipykernel_21265/3757912852.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ins_temp['datetime'] = pd.to_datetime(df_ins_temp['epoch'], unit='s')


# Get daily outside temperature for weather data

In [35]:
# data types
df_weather['temperature_2m'] = df_weather['temperature_2m'].astype(float)

df_out_temp = df_weather.copy()

# rename temperature
df_out_temp.rename(columns={'temperature_2m': 'temperature'}, inplace=True)


# Convert epoch_seconds to datetime
df_out_temp['datetime'] = pd.to_datetime(df_out_temp['time'], unit='s')

# Select only the target column (temperature) and the epoch time
df_target = df_out_temp[['datetime', 'temperature']]

# Set datetime as index
df_target.set_index('datetime', inplace=True)

# Resample by day and calculate the average temperature
df_daily_outside = df_target.resample('D').mean()

# Reset index to make 'datetime' a column again
df_daily_outside.reset_index(inplace=True)


## Get daily average for gas usage

### Drop nans !

In [39]:
# data types
df_p1g['total_gas_used'] = df_p1g['total_gas_used'].astype(float)

df_gas = df_p1g.copy()


# drop NaN
rows_with_nans = df_gas[df_gas['total_gas_used'].isna()]

# Drop rows with NaNs
df_gas = df_gas.dropna(subset=['total_gas_used'])


# Display observations that were dropped
print("\nRows that were dropped:")
print(rows_with_nans)


# Convert epoch_seconds to datetime
df_gas['datetime'] = pd.to_datetime(df_out_temp['time'], unit='s')

# Select only the target column (temperature) and the epoch time
df_target = df_gas[['datetime', 'total_gas_used']]

# Set datetime as index
df_target.set_index('datetime', inplace=True)

# Resample by day and calculate the average temperature
df_daily_gas = df_target.resample('D').mean()

# Reset index to make 'datetime' a column again
df_daily_gas.reset_index(inplace=True)

# no idea why, but for 02-02-2024 a nan was averaged
df_daily_gas = df_daily_gas.dropna(subset=['total_gas_used'])



Rows that were dropped:
          id                 time  total_gas_used
0          1  2022-01-01 01:00:00             NaN
1          2  2022-01-01 01:15:00             NaN
2          3  2022-01-01 01:30:00             NaN
3          4  2022-01-01 01:45:00             NaN
4          5  2022-01-01 02:00:00             NaN
...      ...                  ...             ...
80711  80712  2024-04-22 00:45:00             NaN
80712  80713  2024-04-22 01:00:00             NaN
80713  80714  2024-04-22 01:15:00             NaN
80714  80715  2024-04-22 01:30:00             NaN
80715  80716  2024-04-22 01:45:00             NaN

[8494 rows x 3 columns]


# merge data frames to target data frames

In [42]:

df_merge = pd.merge(df_daily_inside, df_daily_outside, on='datetime', how='inner')
df_final = pd.merge(df_merge, df_daily_gas, on='datetime', how='inner')

df_final.rename(columns={'temperature_x': 'temperature_inside'}, inplace=True)
df_final.rename(columns={'temperature_y': 'temperature_outside'}, inplace=True)